In [96]:
import os
import boto3

In [97]:
os.environ.setdefault('AWS_PROFILE', 'shashank-iam')
os.environ.setdefault('AWS_DEFAULT_REGION', 'us-east-1')

'us-east-1'

In [98]:
cluster_name = 'EMR-Spark-Hive-EBS32'
elastic_ip_name = 'EMR Master IP'

emr_client = boto3.client('emr')
clusters = emr_client.list_clusters(ClusterStates=['RUNNING', 'WAITING'])['Clusters']

In [99]:
cluster_id = ''

for cluster in clusters:
    if cluster['Name'] == cluster_name:
        cluster_id = cluster['Id']

print(cluster_id)

j-1PM7CKA00PQOF


In [100]:
cluster_details = emr_client.describe_cluster(ClusterId=cluster_id)

In [101]:
master_public_dns = cluster_details['Cluster']['MasterPublicDnsName']
print(master_public_dns)

ec2-3-89-159-93.compute-1.amazonaws.com


In [102]:
# master node instance id
master_node_instance_id = ''
instances = emr_client.list_instances(ClusterId=cluster_id)['Instances']
for instance in instances:
    if instance['PublicDnsName'] == master_public_dns:
        master_node_instance_id = instance['Ec2InstanceId']

print(master_node_instance_id)

i-0b63c12d0f4ea08a6


In [103]:
ec2_client = boto3.client('ec2')


In [104]:
# paid elastic ip
allocation_id = ''

elastic_ips = ec2_client.describe_addresses()['Addresses']
for elastic_ip in elastic_ips:
    if elastic_ip['Tags']:
        for tag in elastic_ip['Tags']:
            if tag['Key'] == 'Name' and tag['Value'] == elastic_ip_name:
                allocation_id = elastic_ip['AllocationId']

print(allocation_id)

eipalloc-04e0463716d90d021


In [105]:
print(master_node_instance_id)
print(allocation_id)

ec2_client.associate_address(AllocationId = allocation_id, InstanceId=master_node_instance_id)

i-0b63c12d0f4ea08a6
eipalloc-04e0463716d90d021


{'AssociationId': 'eipassoc-0de18b7d7dc97257d',
 'ResponseMetadata': {'RequestId': '93331709-ca5e-4231-897d-38d18d08422b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '93331709-ca5e-4231-897d-38d18d08422b',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '295',
   'date': 'Sun, 11 Sep 2022 10:36:10 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}